<a href="https://colab.research.google.com/github/amrosnik/adventure_scrape/blob/master/INFINNERATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

InFINNerator: an RNN for generating Finn the Human speech

Here's my first attempt at creating an RNN. I will be using a homemade dataset of Adventure Time transcripts of Finn dialogue to generate "new" Finn speech! 

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Let's first look at the data. I've outlined elsewhere how I formatted the dataset, but for now let's unpack the pickled Finn data.

In [175]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import unicodedata
import string
import pickle
import random
import cloudpickle as cp
from urllib.request import urlopen
import time
import math
import torch.optim as optim

all_letters = string.printable #string.ascii_letters + " .,;'!?-"
n_letters = len(all_letters)+1
url = 'https://github.com/amrosnik/adventure_scrape/raw/master/finn_just_dialogue.pkl'
finn_data = cp.load(urlopen(url)) 

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
finn_text = [unicodeToAscii(line) for line in finn_data]
finn_dialogue = [line for line in finn_text if len(line) > 0 ]
random.shuffle(finn_dialogue)

# concatenate all lines into one big corpus:
finn_corpus = ''.join(str(elem) for elem in finn_dialogue)
#print(len(finn_corpus))
batch_size = round(len(finn_corpus) / 10)
batch_remainder = abs(len(finn_corpus)-batch_size*10) ## add remainder to first batch
#print(batch_size*10)

start_batch = [i*batch_size +i for i in range(10)]
end_batch = [(i+1)*batch_size + i for i in range(10)]
#print(start_batch,end_batch)

torch.manual_seed(1)
## ripped from https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html 

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
# One-hot matrix of first to last letters (not including EOS) for input
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    #print(tensor)
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    #print('before ',letter_indexes)
    #for i in range(len(letter_indexes)):
    #   print(line[i],letter_indexes[i])
    letter_indexes.append(n_letters - 1) # EOS
    #print('after append ',letter_indexes)
    return torch.LongTensor(letter_indexes)
#print(letterToTensor('F'))
#print(lineToTensor('Finn').size())
#print(lineToTensor(finn_dialogue[0]).size())

   # Random item from a list
def randomChoice(l):
      return l[random.randint(0, len(l) - 1)]

# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    line = randomChoice(finn_dialogue)
    input_line_tensor = lineToTensor(line)
    target_line_tensor = targetTensor(line)
    #print(input_line_tensor, target_line_tensor)
    return input_line_tensor, target_line_tensor
  
for i in range(10):
    input_line_tensor, target_line_tensor = randomTrainingExample()
    #print(input_line_tensor.size(), target_line_tensor.size())
  

[0, 29831, 59662, 89493, 119324, 149155, 178986, 208817, 238648, 268479] [29830, 59661, 89492, 119323, 149154, 178985, 208816, 238647, 268478, 298309]


In [0]:
### create batches then one-hot? 
### or one-hot everything then make into batches? 

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        #self.o2o = nn.Linear(hidden_size + output_size, output_size)
        #self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        #output_combined = torch.cat((hidden, output), 1)
        #output = self.o2o(output_combined)
        #output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
            

In [158]:
criterion = nn.NLLLoss()

learning_rate = 0.001
rnn = RNN(n_letters, 128, n_letters)
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

def train(input_line_tensor, target_line_tensor):
    #print('before ',target_line_tensor.size())
    target_line_tensor.unsqueeze_(-1) #OG code: target_line_tensor.unsqueeze_(-1)
    #print(target_line_tensor.size())
    hidden = rnn.initHidden()

    #rnn.zero_grad()
    optimizer.zero_grad()

    #loss = 0
    loss = torch.empty(1)
    #print('before loop',loss)
    output = torch.empty((input_line_tensor.size(1),input_line_tensor.size(2)),requires_grad=True)
    #print(input_line_tensor.size(0))

    #print(target_line_tensor)
    for i in range(input_line_tensor.size(0)):
        #print(i,input_line_tensor.size(),input_line_tensor.size(0))
        output, hidden = rnn(input_line_tensor[i], hidden)
        #print(i,input_line_tensor[i],target_line_tensor[i])
        #print(target_line_tensor[i],target_line_tensor)
        l = criterion(output, target_line_tensor[i])
        loss += l
        #print(l,loss)

    #print('after loop',loss)
    loss.requires_grad_()
    #print('after loop II',loss)
    loss.backward()

    optimizer.step()
    #for p in rnn.parameters():
    #    p.data.add_(-learning_rate, p.grad.data)

    #print(input_line_tensor.size(0), loss.item() / input_line_tensor.size(0))
    return output, loss.item() / input_line_tensor.size(0)
  

#def train_batch
#    n_epochs = 100 # or whatever
#    batch_size = 128 # or whatever

#    for epoch in range(n_epochs):

#        # X is a torch Variable
 #       permutation = torch.randperm(X.size()[0])

#        for i in range(0,X.size()[0], batch_size):
#            optimizer.zero_grad()

 #           indices = permutation[i:i+batch_size]
 #           batch_x, batch_y = X[indices], Y[indices]

            # in case you wanted a semi-full example
 #           outputs = model.forward(batch_x)
 #           loss = lossfunction(outputs,batch_y)

#            loss.backward()
#            optimizer.step()

  
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
  
n_iters = 50000
print_every = 500
plot_every = 50
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss
    #print(loss)
    
    #print('out',loss)
    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0


0m 5s (500 1%) 3.4234
0m 11s (1000 2%) 2.5835
0m 18s (1500 3%) 2.5751
0m 24s (2000 4%) 2.4448
0m 30s (2500 5%) 2.3699
0m 37s (3000 6%) 2.3530
0m 42s (3500 7%) 2.2161
0m 48s (4000 8%) 2.4279
0m 55s (4500 9%) 2.6430
1m 2s (5000 10%) 2.1516
1m 9s (5500 11%) 1.9186
1m 16s (6000 12%) 2.0725
1m 22s (6500 13%) 2.0845
1m 28s (7000 14%) 2.1284
1m 34s (7500 15%) 1.9891
1m 40s (8000 16%) 2.1381
1m 46s (8500 17%) 2.4220
1m 52s (9000 18%) 2.1813
1m 58s (9500 19%) 1.4975
2m 5s (10000 20%) 2.2425
2m 11s (10500 21%) 2.1420
2m 17s (11000 22%) 2.1963
2m 23s (11500 23%) 2.1403
2m 29s (12000 24%) 2.9414
2m 36s (12500 25%) 2.3185
2m 42s (13000 26%) 1.6302
2m 49s (13500 27%) 2.0853
2m 56s (14000 28%) 1.7070
3m 3s (14500 28%) 1.7714
3m 10s (15000 30%) 2.2004
3m 17s (15500 31%) 1.9982
3m 23s (16000 32%) 2.3702
3m 29s (16500 33%) 1.9283
3m 36s (17000 34%) 2.0042
3m 42s (17500 35%) 2.4992
3m 48s (18000 36%) 2.2224
3m 55s (18500 37%) 2.0622
4m 1s (19000 38%) 4.0587
4m 7s (19500 39%) 2.0385
4m 13s (20000 40%) 2.0

In [162]:
max_length = 100

# Sample from a category and starting letter
def sample(start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        input = lineToTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = lineToTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(start_letter))

samples('sds')

s me bed you dont ing the s all the s all the s all the s all the s all the s all the s all the s all
d you dont ing the s all the s all the s all the s all the s all the s all the s all the s all the s 
s me bed you dont ing the s all the s all the s all the s all the s all the s all the s all the s all
